In [57]:
import h5py
import os
import pandas as pd
import json
import numpy as np

# Vorbereitungen

## Dateien in Array schreiben

In [58]:
directory = 'datasetsRosen'
datasets = []

for filename in os.listdir(directory):
    file = h5py.File(directory + '/' + filename)
    for key in file.keys():
        datasets.append(file[key])
        
print(f'{len(datasets)} Datensätze erfolgreich in datasets-Array eingelesen')

2000 Datensätze erfolgreich in datasets-Array eingelesen


## Dateien in Pandas Dataframe schreiben

In [59]:
def read_hdf5(file_path):
    """Read an HDF5 file into a Pandas DataFrame and return it along with attributes."""
    with h5py.File(file_path, 'r') as file:
        for keyname in file.keys():
            if keyname != "":
                data_group_name = keyname
            else:
                print(f"'data' missing in: {file_path}")
                return None, None

        data_group = file[data_group_name]
        data_dict = {name: data_group[name][:] for name in data_group.keys()}

        # Extract attributes
        attributes = {attr: data_group.attrs[attr] for attr in data_group.attrs.keys()}
        data_id = attributes.get('id', 'Unknown')

        max_length = max(len(values) for values in data_dict.values())
        shorter_columns = {name: len(values) for name, values in data_dict.items() if len(values) < max_length}
        if shorter_columns:
            print(f"Inconsistency in: {data_id}")
            for name, length in shorter_columns.items():
                print(f" - Column name:: {name}, Length: {length}")
            return None, attributes

        data_frame = pd.DataFrame(data_dict)
        return data_frame, attributes

all_dataframes = {}
for file_name in os.listdir('datasetsRosen'):
    if file_name.endswith('.h5'):
        file_path = os.path.join('datasetsRosen', file_name)
        frame, frame_attrs = read_hdf5(file_path)
        if frame is not None:
            all_dataframes[file_name] = (frame, frame_attrs)
 print(all_dataframes)

In [60]:
for name, (df, attrs) in all_dataframes.items():
    print(f"{name}: DataFrame Shape: {df.shape}, Attributes: {attrs}")
    print("First 5 lines of the columns:")
    print(df[['velocity', 'defect_channel', 'distance', 'magnetization', 'timestamp', 'wall_thickness']].head())
    print("\n")

000bc7ea-8918-420c-bd13-30e1a6e8dd00.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'Africa', 'id': '000bc7ea-8918-420c-bd13-30e1a6e8dd00', 'instrument': 'Pufferfish'}
First 5 lines of the columns:
   velocity  defect_channel  distance  magnetization     timestamp  \
0  0.774117             0.0  0.000000      -1.918574  8.643687e+08   
1  0.711733             0.0  0.373373       5.128566  8.643692e+08   
2  0.882543             0.0  0.746747       1.558591  8.643697e+08   
3  0.873818             0.0  1.120120       0.140658  8.643702e+08   
4  0.787977             0.0  1.493493       6.028222  8.643707e+08   

   wall_thickness  
0       25.255805  
1       29.027724  
2       24.149902  
3       22.791865  
4       31.121774  


006b0b34-324b-496a-8ed7-a40f7d689ee9.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'Europe', 'id': '006b0b34-324b-496a-8ed7-a40f7d689ee9', 'instrument': 'Unicorn'}
First 5 lines of the columns:


KeyError: "['velocity'] not in index"

# Datenbereinigung die Zweite

In [62]:
def get_dataframe_from_directory(directory):
    dataframes = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.h5'):
            file_path = os.path.join(directory, file_name)
            frame, frame_attrs = read_hdf5(file_path)
            if frame is not None:
                dataframes[file_name] = (frame, frame_attrs)
    return dataframes
    
def check_dataframe(dataframes):
    newFrame = pd.DataFrame()
    
    for frame in dataframes.values():
        numeric = drop_not_numerical_rows(frame[0])
        newFrame = newFrame.append(numeric)
    return newFrame

def drop_not_numerical_rows(dataframe):
    numeric_df = dataframe.apply(pd.to_numeric, errors='coerce')
    numeric_df = numeric_df.dropna()
    return numeric_df
      
df = check_dataframe(get_dataframe_from_directory('datasetsRosen'))

save_dataframe_to_json(df, 'generated files')

# Datenbereinigung !!ChatGPT und kacke!!

In [ ]:
import os
import h5py
import numpy as np

def check_file(filename):
    try:
        with h5py.File(filename, 'r') as file:
            # Überprüfe hier die Struktur und Integrität der Daten
            check_dataset(file, 'defect_channel')
            check_dataset(file, 'distance')
            check_dataset(file, 'magnetization')
            check_dataset(file, 'timestamp')
            check_dataset(file, 'velocity')
            check_dataset(file, 'wall_thickness')
    except Exception as e:
        print(f"Fehler beim Lesen von {filename}: {e}")

def check_dataset(file, dataset_name):
    try:
        dataset = file['data'][dataset_name][:]
        validate_numerical_data(dataset)
        check_missing_data(dataset)
        # Weitere spezifische Überprüfungen für jedes Dataset können hinzugefügt werden
    except KeyError:
        print(f"Fehler: Dataset '{dataset_name}' nicht vorhanden.")
    except Exception as e:
        print(f"Fehler bei der Überprüfung von '{dataset_name}': {e}")

def validate_numerical_data(dataset):
    # Beispiel: Überprüfe, ob alle Werte positiv sind
    if (dataset < 0).any():
        print("Fehler: Negative Werte in den Daten.")

# Weitere Validierungsbedingungen können hier hinzugefügt werden

def check_missing_data(dataset):
    # Beispiel: Überprüfe, ob NaN-Werte in den Daten vorhanden sind
    if np.isnan(dataset).any():
        print("Fehler: NaN-Werte in den Daten.")

# Weitere Überprüfungen auf fehlende Daten können hier hinzugefügt werden

def log_result(filename, result):
    with open("log.txt", "a") as log_file:
        log_file.write(f"{filename}: {result}\n")

# Iteriere über alle Dateien und prüfe jede einzelne
folder_path = "datasetsRosen"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".h5"):
        check_file(os.path.join(folder_path, file_name))


# Datenüberprüfung

## Welche Konfigurationen und Instrumente gibt es in den Datensätzen?

In [ ]:
confs_directory = 'datasetsRosen'
configurations = []
instruments = []
configAmount = 0
instAmount = 0

for idx, filename in enumerate(os.listdir(confs_directory)):
    file = h5py.File(confs_directory + '/' + filename)
    if 'data' in file:
        configName = file['data'].attrs['configuration']
        if file['data'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['data'].attrs['instrument']
        if file['data'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)
    if 'Daten' in file:
        configName = file['Daten'].attrs['configuration']
        if file['Daten'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['Daten'].attrs['instrument']
        if file['Daten'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)

print(f'Menge an Configurations: {configAmount}')
print('Configurations: ', configurations)
print(f'Menge an Instruments: {instAmount}')
print('Instruments: ', instruments)

## Wie viele falsche / fehlende Datensätze gibt es?

Prüft auf: 
    Negative Werte,
    Fehlende Datensätze / Falsche Key Bezeichnung

In [ ]:
# defect_channel, distance, magnetization, timestamp, velocity, wall_thickness
attribute = ("magnetization"
             "") # attribute ersetzen um verschiedene Attribute zu testen
count_negativeValue = 0
negativeValuesIdx = []
count_noDataset = 0
noDatasetIdx = []
eastereggCount = 0

for idx, dataset in enumerate(datasets):
    # print(dataset[attribute][...])
    if attribute in dataset:
        if b'Easteregg :)' not in dataset[attribute][...]:
            velocity_data = dataset[attribute][...].astype(float)
            if (velocity_data < 0).any():
                count_negativeValue += 1
                negativeValuesIdx.append(idx)
        else:
            eastereggCount += 1
    else:
        count_noDataset += 1
        noDatasetIdx.append(idx)
        
print(f"---------------------------------------------- {attribute} Analyse ----------------------------------------------")
print(f"Anzahl negative Werte: {count_negativeValue}\n Indizes: {negativeValuesIdx}\n")
print(f"Anzahl fehlender {attribute} Datensätze: {count_noDataset}\n Indizes: {noDatasetIdx}")
print("--------------------------------------------------------------------------------------------------------------")
print(f"Anzahl Eastereggs: {eastereggCount}")
print(datasets[1959].keys())

## Pandas Dataframe als Dateien speichern

In [ ]:
def save_dataframe_to_json(dataframes, output_directory):
    json_string = dataframes.to_json(orient="table")
    file_path = os.path.join(output_directory, 'data' + ".json")

    with open(file_path, 'w') as file:
        file.write(json_string)